## CA 3, LLMs Spring 2024

- **Name:** Majid Faridfar
- **Student ID:** 810199569

---

# Chain-of-Thought (CoT) (20 points)

LLMs have demonstrated good reasoning abilities. Furthermore, their capabilities can be further improved by incorporating reasoning techniques. One of the most notable developments in this area is the [Chain-of-Thought (CoT)](https://arxiv.org/abs/2201.11903), which was introduced by Google. This approach has shown promising results in improving the reasoning capabilities of language models across a variety of tasks. Can you explain what CoT is and how it works? (2.5 Points)

> Unlike traditional input-output prompting, which involves asking a single question and receiving a direct answer, CoT encourages LLMs to think logically and sequentially by breaking down complex tasks into intermediate steps. When faced with a challenging problem (such as complex arithmetic, commonsense reasoning, or symbolic tasks), CoT prompts LLMs to break down their response into smaller, more manageable steps. Just as humans naturally decompose complex problems, CoT provides LLMs with a roadmap to follow.
>
> CoT also achieves few-shot learning by providing the LLM with a few examples that demonstrate the reasoning process. These examples guide the model through intermediate steps, allowing it to arrive at a final answer. Importantly, CoT doesn't require adjusting model weights, it can be done in-context without retraining.
>
> An additional benefit of CoT is that it helps us understand and debug LLMs' reasoning deficits. By explicitly outlining the reasoning process, CoT enables more accurate and reliable outputs.

In this section, you should use the CoT technique. firstly you need to load the [Phi-2 model](https://www.microsoft.com/en-us/research/blog/phi-2-the-surprising-power-of-small-language-models/). This model has been introduced by Microsoft as a small LLM

In [ ]:
device = 'cuda'

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device(device)

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiSdpaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((256

In [ ]:
def generate_output(model, input, max_length=300):
  input = f"Question: {input}\nOutput:"
  input = tokenizer(input, return_tensors="pt", return_attention_mask=False)
  input.to(device)
  outputs = model.generate(**input, max_length=max_length)
  text = tokenizer.batch_decode(outputs)[0]
  return text

Use Phi-2 to answer the questions below with and without CoT. Compare results and explain their difference. (4 Points)

In [ ]:
questions = ["Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
"Jack is stranded on a desert island. He wants some salt to season his fish. He collects 2 liters of seawater in an old bucket. If the water is 20% salt, how many ml of salt will Jack get when all the water evaporates?",
"John volunteers at a shelter twice a month for 3 hours at a time. How many hours does he volunteer per year?",
"There are 32 tables in a hall. Half the tables have 2 chairs each, 5 have 3 chairs each and the rest have 4 chairs each. How many chairs in total are in the hall?",
"Bert fills out the daily crossword puzzle in the newspaper every day. He uses up a pencil to fill out the puzzles every two weeks. On average, it takes him 1050 words to use up a pencil. How many words are in each crossword puzzle on average?"
]

In [ ]:
# WRITE YOUR CODE HERE

answers_without_cot = []
for question in questions:
    answer = generate_output(model, question)
    answers_without_cot.append(answer)

In [ ]:
answers_with_cot = []
for question in questions:
    question += " Let's think step by step."
    answer = generate_output(model, question)
    answers_with_cot.append(answer)

In [ ]:
print("--------------------------------------------")
for i in range(len(questions)):
    print(f"[{i}]\n")
    print("**Question**\n" + questions[i])
    print("\n**Answer without CoT**\n" + answers_without_cot[i])
    print("\n**Answer with CoT**\n" + answers_with_cot[i])
    print("\n--------------------------------------------")

--------------------------------------------
[0]

**Question**
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?

**Answer without CoT**
Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Output: Weng earned $9 for babysitting.
<|endoftext|>

**Answer with CoT**
Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn? Let's think step by step.
Output: To find out how much Weng earned, we need to convert the 50 minutes into hours. Since there are 60 minutes in an hour, we divide 50 by 60 to get the decimal equivalent. 50/60 = 0.83. Now, we can multiply the decimal by Weng's hourly rate of $12. 0.83 x 12 = $9.96. Therefore, Weng earned $9.96 for babysitting.
<|endoftext|>

--------------------------------------------
[1]

**Question**
Jack is stranded on a desert island. He wants some sa

>**Analysis**
>- Question $0$: Weng earns \$12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
>  - Correct answer: 50/60 * 12 = 10
>  - Without CoT answer: 9
>    - It's obviously incorrect. Also, no explanation is given.
>  - With CoT answer: 9.96
>    - It's very close to the correct answer. But the solution is compeletely correct and the reason behind this little error is also provided which is due to estimating 50/60 to 0.83.
> - Question $1$: Jack is stranded on a desert island. He wants some salt to season his fish. He collects 2 liters of seawater in an old bucket. If the water is 20% salt, how many ml of salt will Jack get when all the water evaporates?
>  - Correct answer: 20/100 \* 2000 = 400
>  - Without CoT answer: 400
>    - It's correct. The solution was not wanted, but the provided one is correct.
>  - With CoT answer: 400
>    - It's correct. Also, the solution is compeletely correct.
> - Question $2$: John volunteers at a shelter twice a month for 3 hours at a time. How many hours does he volunteer per year?
>  - Correct answer: 12\*2\*3 = 72
>  - Without CoT answer: 36
>    - It's incorrect. The provided solution also looks like holucination.
>  - With CoT answer: 36
>    - It's correct. Also based on the solution, we can understand there is a little mistake in calculation which is related to not considering 'twice a month' in question. The point here is that we know where the problem exactly is, as a result it is easier to address and debug.
> - Question $3$: There are 32 tables in a hall. Half the tables have 2 chairs each, 5 have 3 chairs each and the rest have 4 chairs each. How many chairs in total are in the hall?
>  - Correct answer: (32/2)\*2 + 5\*3 + (32-((32/2)+5))*4 = 91
>  - Without CoT answer: 121
>    - It's incorrect. The solution was not wanted, but in the provided one, we can see a little mistake (instead of 5\*3 the model has written 15\*3).
>  - With CoT answer: 91
>    - It's correct. Also, the solution is compeletely correct.
> - Question $4$: Bert fills out the daily crossword puzzle in the newspaper every day. He uses up a pencil to fill out the puzzles every two weeks. On average, it takes him 1050 words to use up a pencil. How many words are in each crossword puzzle on average?
>  - Correct answer: 1050/(2\*7) = 75
>  - Without CoT answer: 75
>    - It's correct. But the provided solution also looks like holucination.
>  - With CoT answer: 75
>    - It's correct. Also, the solution is compeletely correct.
>
> Overall, CoT (here I have used Zero-shot CoT) has improved the performance of model and makes it generate more accurate answers, though there are some cases (such as question 4) that not using CoT results in correct answers as well, but CoT didn't destroy it. However there are some cases (such as question 2), it couldn't help but at least it helps us understand where the problem is, so we can solve it more easily.

## Other Methods for Reasoning

There are many other approaches to utilize the reasoning abilities of LLMs. Describe the [Tree-of-Thought (ToT)](https://arxiv.org/abs/2305.10601) and [Self-Consistency](https://arxiv.org/abs/2203.11171) within these approaches. (3.5 Points)

> ToT is framework for language model inference that goes beyond the traditional left-to-right decision-making process. It enables exploration over coherent units of text (referred to as "thoughts") as intermediate steps toward problem-solving. LLMs using ToT consider multiple reasoning paths and self-evaluate choices to decide the next course of action. Also ToT allows LLMs to look ahead or backtrack when necessary, making global decisions rather than being confined to token-level decisions. ToT significantly enhances LLMs' problem-solving abilities on tasks requiring non-trivial planning or search, such as the Game of 24, Creative Writing, and Mini Crosswords
>
> Self-Sonsistency also known as CoT-SC is an ensemble approach that builds upon the Chain of Thought (CoT) method. CoT-SC samples multiple independent chains of thought and returns the most frequent output. It improves upon CoT by considering different reasoning paths and achieving better performance.
>
> Note: AI helped me answering this question by explaining these methods.


Now, implement Self-Consistency to answer the questions of the previous section. (6 Points)

In [ ]:
import re

# Assume that the last number in the response is final answer.
def get_numeric_answer(answer_str):
    return re.findall(r"[-+]?(?:\d*\.*\d+)", answer_str)[-1]

In [ ]:
# This function is AI-generated
def most_frequent(List):
    counter = 0
    num = List[0]

    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i

    return num

In [ ]:
# WRITE YOUR CODE HERE

PATHS = 5
answers = []
questions_paths = []

i = 0
for question in questions:
    q_answers = []
    answers_votes = []
    for i in range(PATHS):
        model.train() # To reset model to generate different responses for same questions

        answer = generate_output(model, question + " Let's think step by step.")
        answer_vote = get_numeric_answer(answer)

        q_answers.append(answer)
        print(answer)

        answers_votes.append(answer_vote)

    questions_paths.append(q_answers)
    answers.append(most_frequent(answers_votes))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn? Let's think step by step.
Output: To find out how much Weng earned, we need to calculate the total amount of money she made for the 50 minutes of babysitting. Since she earns $12 an hour, we can divide the total amount of money by the number of hours she worked. 

First, we need to convert the 50 minutes into hours. Since there are 60 minutes in an hour, we can divide 50 by 60 to get the fraction of an hour. 

50 minutes ÷ 60 minutes/hour = 5/6 hour

Now, we can multiply the fraction of an hour by the hourly rate to find the total amount of money earned. 

5/6 hour × $12/hour = $10

Therefore, Weng earned $10 for the 50 minutes of babysitting.
<|endoftext|>


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn? Let's think step by step.
Output: To find out how much Weng earned, we need to calculate the total amount of money she earned for the 50 minutes of babysitting. Since she earns $12 an hour, we can divide the total amount of money she earned by the number of hours she worked. 

First, we need to convert the 50 minutes into hours. Since there are 60 minutes in an hour, we divide 50 by 60 to get the number of hours worked. 

50 minutes ÷ 60 minutes/hour = 0.83 hours

Next, we multiply the number of hours worked by the hourly rate to find the total amount of money earned. 

0.83 hours × $12/hour = $10.04

Therefore, Weng earned $10.04 for the 50 minutes of babysitting.
<|endoftext|>


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Question: Bert fills out the daily crossword puzzle in the newspaper every day. He uses up a pencil to fill out the puzzles every two weeks. On average, it takes him 1050 words to use up a pencil. How many words are in each crossword puzzle on average? Let's think step by step.
Output: Step 1: Calculate the number of pencils Bert uses in a year. Since he uses a pencil every two weeks, he uses a pencil 52/2 = 26 times in a year.
Step 2: Calculate the total number of words Bert writes in a year. Since he uses 1050 words per pencil, he writes 1050 x 26 = 27,300 words in a year.
Step 3: Calculate the average number of words in each crossword puzzle. Since he fills out the puzzle every day, he fills out 365 puzzles in a year. Therefore, the average number of words in each crossword puzzle is 27,300 / 365 = 75.
<|endoftext|>


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: Bert fills out the daily crossword puzzle in the newspaper every day. He uses up a pencil to fill out the puzzles every two weeks. On average, it takes him 1050 words to use up a pencil. How many words are in each crossword puzzle on average? Let's think step by step.
Output: Step 1: Determine the number of pencils used in a year. Since there are 52 weeks in a year, and Bert fills out the puzzle every two weeks, he uses 52/2 = 26 pencils in a year.
Step 2: Determine the total number of words used in a year. Since it takes 1050 words to use up a pencil, the total number of words used in a year is 26 * 1050 = 27,300 words.
Step 3: Determine the average number of words in each crossword puzzle. Since there are 27,300 words in a year and Bert fills out the puzzle every day, the average number of words in each crossword puzzle is 27,300/365 = 75.
<|endoftext|>


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: Bert fills out the daily crossword puzzle in the newspaper every day. He uses up a pencil to fill out the puzzles every two weeks. On average, it takes him 1050 words to use up a pencil. How many words are in each crossword puzzle on average? Let's think step by step.
Output: To find the average number of words in each crossword puzzle, we need to divide the total number of words used up by the number of puzzles filled out.

Step 1: Calculate the number of puzzles filled out in two weeks.
Since Bert fills out the crossword puzzle every day, in two weeks, he fills out 14 puzzles (2 weeks x 7 days/week).

Step 2: Calculate the total number of words used up in two weeks.
Since it takes him 1050 words to use up a pencil, in two weeks, he uses up 1050 x 14 = 14,700 words.

Step 3: Calculate the average number of words in each crossword puzzle.
To find the average, we divide the total number of words used up (14,700) by the number of puzzles filled out (14).
Average number of words

In [ ]:
print("--------------------------------------------")
for i in range(len(questions)):
    print(f"[{i}]\n")
    print("**Question**\n" + questions[i])
    print("\n**Answer**\n" + answers[i])
    print("\n**Reasoning Paths**")
    print('++++\n\n'.join(questions_paths[i]))
    print("\n--------------------------------------------")

--------------------------------------------
[0]

**Question**
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?

**Answer**
50

**Reasoning Paths**
Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn? Let's think step by step.
Output: To find out how much Weng earned, we need to calculate the total amount of money she made for the 50 minutes of babysitting. Since she earns $12 an hour, we can divide the total amount of money by the number of hours she worked. 

First, we need to convert the 50 minutes into hours. Since there are 60 minutes in an hour, we can divide 50 by 60 to get the fraction of an hour. 

50 minutes ÷ 60 minutes/hour = 5/6 hour

Now, we can multiply the fraction of an hour by the hourly rate to find the total amount of money earned. 

5/6 hour × $12/hour = $10

Therefore, Weng earned $10 for the 50 minutes of babysitting.
<|endoftext

Consider LLMs' features and propose a new approach based on them to enhance LLMs' reasoning abilities. Why do you believe this approach could enhance LLMs' reasoning abilities? (4 Points)

> Several strategies and techniques can lead to significant advancements. However one novel approach is Self-Reinforcement with Weak Supervision. Since LLMs often rely on extensively annotated datasets for fine-tuning, which can be resource-intensive and challenging to scale, we need an approach that leverages minimal human supervision. The methodology is as follows:
>
> 1. Begin with Supervised Fine-Tuning (SFT) using a small set of annotated questions.
> 2. Iteratively improve LLMs by learning from differences between responses from SFT and unfinetuned models on unlabeled questions.
> 3. Self-reinforcement encourages LLMs to explore alternative reasoning paths and adapt without relying heavily on human-annotated explanations.
>
> Pros:
> - Efficiency: Fewer annotated examples are required.
> - Scalablity: Larger models and data requirements are possible to handle.
>
> An example Application is PuzzleBen benchmark comprising complex questions, answers, and rationales across various domains (brainteasers, puzzles, riddles, etc.) with both annotated and unannotated questions.
>
> Note: I asked this question from ChatGPT.